In [1]:
import pandas as pd
from Constraint import Problem, Constraint
from Nature import attribute_nature

In [2]:
experiment_constraint_data = pd.read_csv('./experiment_constraint_data.csv')

In [3]:
experiment_constraint_data.head(10)

,name,value,priority,constraint_no
0,ram,64,3,0
1,screen_size,13.71,4,0
2,storage,512,5,0
3,cpu,i7,5,0
4,os,10,5,0
5,screen_size,15.6,1,1
6,brand,microsoft,4,1
7,cpu,pentium,3,1
8,storage,1024,2,1
9,price,2368.59,1,1


In [4]:
problem_list = list()
unique_constraint_no = experiment_constraint_data['constraint_no'].unique()
for no in unique_constraint_no:
    problem: Problem = Problem()
    constraint_list = list()
    for index, item in experiment_constraint_data[experiment_constraint_data['constraint_no'] == no].iterrows():
        constraint = None
        if item.get('name') in ['brand', 'cpu','gpu','os']:
            constraint: Constraint = Constraint(item.get('name'), item.get('value'), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['storage', 'ram','price','weight']:
            constraint: Constraint = Constraint(item.get('name'), float(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['camera']:
            constraint: Constraint = Constraint(item.get('name'), bool(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        if constraint != None:
            constraint_list.append(constraint)
    cl_set = set(constraint_list)
    problem.constraint_list = set(constraint_list)
    problem_list.append(problem)

In [5]:
def get_df_len(df):
    if df is None:
        return 0
    return (df.shape)[0]

for problem in problem_list:
    print(get_df_len(problem.solve()))

0
0
0
0
10
25
5
0
0
3
0
10
0
2
0
0
1
20
19
19
0
0
0
0
0
1
0
0
0
0
0
3
28
0
2
0
0
0
87
0
9
4
0
0
13
34
0
2
0
19
0
0
8
0
0
0
19
0
4
0
0
1
0
0
6
1
0
3
2
0
6
0
3
0
9
0
6
0
0
0
0
0
20
0
0
0
2
0
0
13
0
0
0
0
0
1
0
0
3
1
4
0
22
6
0
1
1
0
0
7
0
0
0
41
45
0
16
2
0
14
0
0
0
15
3
0
0
0
0
0
0
0
5
5
23
0
0
